# Задание 3

#### CounterGetter (0.1 балл)
Реализуйте класс CounterGetter, который на вход принимает произвольное количество именованых аргументов и считает количество обращений к полям и методам класса во всех экземплярах.

## Реализация

In [1163]:
from collections import Counter

class CounterGetter(object):
    __counter = Counter()
    
    def __init__(self, **kwargs):
        for key in kwargs:
            self.__setattr__(key, kwargs[key])
    
    def __getattribute__(self, name):
        counter = object.__getattribute__(self, "_CounterGetter__counter")
        counter[name] += 1
        object.__setattr__(self, "__counter", counter)

        return object.__getattribute__(self, name)

## Тестирование

In [1164]:
class A(CounterGetter):
    def __init__(self, a, b, c, d):
        self.a = a
        self.mas = [b, c, d]
        
    def sum(self):
        return sum(self.mas)

In [1165]:
a = A(4, 3, 5, 4)

In [1166]:
a.__counter

Counter({'__counter': 1})

In [1167]:
a.sum()

12

In [1168]:
a.__counter

Counter({'__counter': 2, 'mas': 1, 'sum': 1})

In [1169]:
for i in range(1000):
    a.a
a.__counter

Counter({'__counter': 3, 'a': 1000, 'mas': 1, 'sum': 1})

In [1170]:
s = 0
for i in range(len(a.mas)):
    s += a.mas[i]
print(s)
a.__counter

12


Counter({'__counter': 4, 'a': 1000, 'mas': 5, 'sum': 1})

In [1171]:
class B(CounterGetter):
    def __init__(self, f1, f2):
        self.f1 = f1
        self.f2 = f2

In [1172]:
b = B(1, 1)
b.__counter

Counter({'__counter': 5, 'a': 1000, 'mas': 5, 'sum': 1})

In [1173]:
for i in range(99):
    b.f1, b.f2 = b.f2, b.f1 + b.f2
print(b.f1, b.f2)
b.__counter

354224848179261915075 573147844013817084101


Counter({'__counter': 6, 'a': 1000, 'f1': 100, 'f2': 199, 'mas': 5, 'sum': 1})

#### Vector (0.4 балла)

Реализуйте класс вектор, который должен обладать следующими свойствами:
* Над экземплярами выполняются арифметические операции (+-\*/). Операции могут выполняться как с числами, так и с векторами. Если второй операнд вектор, то верните их скалярное произведение, если число, выполните поэлементное действие.
* Реализуйте доступ к элементам вектора по индексам:

<code>vec = Vector(1, 2, 3)

vec[0]</code>
* Реализуйте умножение вектора на матрицу (Матрица задается двумерным списком).
* Поддерживайте методы push_back, pop_back, insert.
* Реализуйте поддержку функции len

P. S. Запрещается использовать numpy и другие библиотеки по работе с векторами. Рекомендуется представлять элементы вектора как список.

## Реальзация

In [1175]:
import math

class Vector(object):
    def __init__(self, *args):        
        if len(args) == 0:
            self.vec = []
        elif len(args) == 1 and isinstance(args[0], list):
            self.vec = args[0]
        else:
            self.vec = list(args)
    
    # 1. Операции над вектором
    def oper(self, value, func):
        if isinstance(value, Vector):
            if len(value) != len(self):
                raise ArithmeticError()
            
            return Vector(*list(map(lambda pair: func(pair[0], pair[1]), 
                                    zip(self.vec, value.vec))))
        else:
            return Vector(*list(map(lambda pair: func(pair[0], pair[1]), 
                                    zip(self.vec, [ value ] * len(self)))))
    
    def __add__(self, value):
        return self.oper(value, lambda x, y: x + y)
    
    def __sub__(self, value):
        return self.oper(value, lambda x, y: x - y)
    
    def __truediv__(self, value):
        return self.oper(value, lambda x, y: x / y)
        
    def __mul__(self, value):
        if isinstance(value, Vector):
            if len(value) != len(self):
                raise ArithmeticError()
                
            result = 0
            for i, v in enumerate(self.vec):
                result += v * value.vec[i]
            return result
        else:
            return self.oper(value, lambda x, y: x * y)
    
    # 2. Досутп к вектору по индексам
    def __setitem__(self, key, value):
        self.vec[key] = value
        
    def __getitem__(self, key):
        return self.vec[key]
    
    # 3. Умножение вектора на матрицу
    def matrix_mult(self, matrix):
        result = Vector([ 0 ] * len(self))
        for j in range(len(matrix[0])):
            column = []
            for i in range(len(matrix)):
                column.append(matrix[i][j] * self.vec[j])
                
            result = result + Vector(column)
        return result
    
    # 4. Поддержка методов push_back, pop_back, insert.
    def push_back(self, element):
        self.vec.append(element)
        
    def pop_back(self):
        return self.vec.pop()
    
    def insert(self, element, index):
        self.vec.insert(element, index)
        
    # 5. Реальзация функции len
    def __len__(self):
        return len(self.vec)
    
    # Остальное
    def __repr__(self):
        return repr(self.vec)
    
    def __str__(self):
        return str(self.vec)

## Тестирование

### Арифметические операции

In [1176]:
v1 = Vector(1, 2, 3, 4, 5)
v2 = Vector(5, 6, 7, 8, 9)

In [1187]:
v1 / 3 + (v2 - 8)

[-2.6666666666666665, -1.3333333333333335, 0.0, 1.3333333333333333, 2.666666666666667]

In [1183]:
v1 * 5 - v2

[0, 4, 8, 12, 16]

In [1185]:
v1 / v2 + 100

[100.2, 100.33333333333333, 100.42857142857143, 100.5, 100.55555555555556]

In [1180]:
v1 * v2

115

In [1188]:
Vector(1, 1, 1, 1, 1) * Vector(1, 1, 1, 1, 1)

5

### Доступ к элементам по индексам

In [1201]:
v1 = Vector(1, 2, 3, 4, 5)
v2 = Vector(5, 6, 7, 8, 9)

In [1202]:
for i in range(len(v1)):
    print(v1[i], end=" ")

1 2 3 4 5 

In [1203]:
print(v1)
for i in range(len(v1)):
    v1[i] = v2[i]
v1, v2

[1, 2, 3, 4, 5]


([5, 6, 7, 8, 9], [5, 6, 7, 8, 9])

### Умножение вектора на матрицу

In [1204]:
v1 = Vector(1, 2, 3, 4, 5)

In [1206]:
v1.matrix_mult([[1, 0, 0, 0, 0], [1, 1, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 1, 0], [1, 1, 1, 1, 1]])

[1, 3, 6, 10, 15]

### Методы `push_back`, `pop_back` и `insert`

In [1221]:
v1 = Vector(1, 2, 3, 4, 5)

In [1222]:
v1.push_back(6)
v1

[1, 2, 3, 4, 5, 6]

In [1223]:
print(v1.pop_back())
v1

6


[1, 2, 3, 4, 5]

In [1224]:
for i in range(len(v1)):
    v1.insert(2 * i, i ** 2)
v1

[0, 1, 1, 2, 4, 3, 9, 4, 16, 5]

### Поддержка функции `len`

In [1235]:
v1 = Vector()

In [1236]:
for i in range(30):
    v1.push_back(i)
    print(len(v1), end=' ')

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 

#### Table (0.5 баллов)

Реализуйте класс для работы с таблицами. Класс должен обладать следующей функциональностью:

* Таблица может задаваться как двумерным списком, так и читаться из файла. Если читается из файла, то должен быть указан разделитель.

* Обладать возможностью задания названия колонок (по номеру, если не задано). Если читается из файла и задан параметр наличия именованных колонок, то первая строчка файла отвечает за названия колонок.

* Реализуйте доступ к колонкам по названю и по индекс:

<code>t = Table()

t["index"]</code>

* Реализуйте функции head и tail, которые показывают заданное число строк с начала и с конца соответственно.
* Реализуйте методы unique и count, которые выводят все уникальные элементы для таблицы или для заданного столбца:

<code>

t["sex"].unique()

t.unique()

</code>

## Реализация

In [973]:
class Table:
    # 1. Илициализация (двумерный список, словарь)
    def __init__(self, data, names=None, inds=None, transposed=False):
        if not transposed:
            self.data = self.transpose(data)
        else:
            self.data = data
        
        if len(self.data) > 0:
            self.rows_num = len(self.data[0])
        else:
            self.rows_num = 0
        
        columns_num = len(self.data)
        
        for column_data in self.data:
            if len(column_data) != self.rows_num:
                raise ValueError("arrays must all be same length")
        
        if names is None:
            names = range(1, columns_num + 1)

        self.__names = {}
        for i, name in enumerate(names):
            self.__names[str(name)] = i
        
        if inds == None:
            self.inds = list(range(1, self.rows_num + 1))
        else:
            self.inds = inds
            
        if len(self.inds) != self.rows_num:
            raise ValueError("arrays must all be same length")            
        
    def transpose(self, data):
        if len(data) > 0:
            columns_num = len(data[0])
        else:
            columns_num = 0
        rows_num = len(data)
        
        transposed_data = []
        for j in range(columns_num):
            column_data = []
            for i in range(rows_num):
                column_data.append(data[i][j])
            transposed_data.append(column_data)
            
        return transposed_data
    
    @property
    def names(self):
        return list(map(lambda pair: pair[0], sorted(list(self.__names.items()), key=lambda x: x[1])))
    
    # 1. Инициализация (считывание из файла)
    @staticmethod
    def read_csv(filename, header=True, sep=";", names=None):
        with open(filename, "r") as file:
            data = list(map(lambda line: line.strip().split(sep), file.readlines()))
            
            if header:
                if names is None:
                    names = data[0]
                data = data[1:]
            
            return Table(data, names=names)
    
    # 2. Возможность переименования столбцов
    def rename(self, old, new):
        old, new = str(old), str(new)
        if new in self.__names:
            raise ValueError("all names must be unique")
        self.__names[new] = self.__names[old]
        self.__names.pop(old)
        
    # 3. Доступ к колонкам по названию и номеру
    def __getitem__(self, key):
        if isinstance(key, str):
            index = self.__names[key]
            return Table([self.data[index]], transposed=True, 
                         names=[key], inds=self.inds)
        else:
            return Table([self.data[key - 1]], transposed=True, 
                         names=[self.names[key - 1]], inds=self.inds)
    
    def __setitem__(self, key, column_data):
        if len(column_data) != self.rows_num:
            raise ValueError("arrays must all be same length") 
            
        if isinstance(key, str):
            self.data[self.__names[key]] = column_data
        else:
            self.data[key - 1] = column_data
    
    # 4. head и tail
    def head(self, num=5):
        new_data = []
        for column_data in self.data:
            new_data.append(column_data[:num])
        
        return Table(new_data, transposed=True, 
                     inds=self.inds[:num], names=self.names)
            
    def tail(self, num=5):
        new_data = []
        for column_data in self.data:
            new_data.append(column_data[max(self.rows_num - num, 0):])
        return Table(new_data, transposed=True, 
                     inds=self.inds[max(self.rows_num - num, 0):], names=self.names)
    
    def __str__(self):
        result = ""
        simple = [ [""] + self.inds ]
        for index, name in enumerate(self.names):
            simple.append([ "'" + name + "'"] + self.data[index])
        
        for j in range(len(simple[0])):
            for i in range(len(simple)):
                result += str(simple[i][j])
                result += "\t"
            result += "\n"
        return result[:-1]
    
    def __repr__(self):
        return str(self)
    
    # 5. unique и count
    def unique(self):
        to_unique = []
        for column in self.data:
            to_unique.extend(column)
        return list(set(to_unique))
    
    def count(self):
        new_data = []
        for column_data in self.data:
            new_data.append(sum(map(lambda x: x is not None, column_data)))
        return Table([ new_data ], transposed=True, names=['count'], inds=self.names)
    
    
    # Остальное
    def __len__(self):
        return self.rows_num
    
    def __iter__(self):
        if len(self.data) > 1:
            data = self.transpose(self.data)
            for row_data in data:
                yield row_data
        else:
            for elem in self.data[0]:
                yield elem

## Тестирование

### Обычная инициализация

In [974]:
Table([[ 1,  2,  3,  4],
       [ 5,  6,  7,  8],
       [ 9, 10, 11, 12],
       [13, 14, 15, 16],
       [17, 18, 19, 20]])

	'1'	'2'	'3'	'4'	
1	1	2	3	4	
2	5	6	7	8	
3	9	10	11	12	
4	13	14	15	16	
5	17	18	19	20	

In [975]:
Table([[ 1,  2,  3,  4],
       [ 5,  6,  7,  8],
       [ 9, 10, 11, 12],
       [13, 14, 15, 16],
       [17, 18, 19, 20]], names=['a', 'b', 'c', 'd'])

	'a'	'b'	'c'	'd'	
1	1	2	3	4	
2	5	6	7	8	
3	9	10	11	12	
4	13	14	15	16	
5	17	18	19	20	

In [976]:
Table([[ 1,  2,  3,  4],
       [ 5,  6,  7,  8],
       [ 9, 10, 11, 12],
       [13, 14, 15, 16],
       [17, 18, 19, 20]], inds=list(range(11, 16)))

	'1'	'2'	'3'	'4'	
11	1	2	3	4	
12	5	6	7	8	
13	9	10	11	12	
14	13	14	15	16	
15	17	18	19	20	

In [977]:
Table([[ 1,  2,  3,  4],
       [ 5,  6,  7,  8],
       [ 9, 10, 11, 12],
       [13, 14, 15, 16],
       [17, 18, 19, 20]], transposed=True)

	'1'	'2'	'3'	'4'	'5'	
1	1	5	9	13	17	
2	2	6	10	14	18	
3	3	7	11	15	19	
4	4	8	12	16	20	

### Считывания из файла

In [978]:
Table.read_csv("table.txt")

	'name'	'test'	'head'	
1	2	3	4	
2	5	4	3	
3	2	3	4	
4	2	3	2	
5	1	3	1	
6	1	1	0	

In [979]:
Table.read_csv("table.txt", sep='\t')

	'name;test;head'	
1	2;3;4	
2	5;4;3	
3	2;3;4	
4	2;3;2	
5	1;3;1	
6	1;1;0	

In [980]:
Table.read_csv("tab_table.txt", sep='\t')

	'name'	'test'	'head'	
1	2	3	4	
2	5	4	3	
3	2	3	4	
4	2	3	2	
5	1	3	1	
6	1	1	0	

### Переименовывание столбцов

In [981]:
t = Table([[ 1,  2,  3,  4],
           [ 5,  6,  7,  8],
           [ 9, 10, 11, 12],
           [13, 14, 15, 16],
           [17, 18, 19, 20]])
t

	'1'	'2'	'3'	'4'	
1	1	2	3	4	
2	5	6	7	8	
3	9	10	11	12	
4	13	14	15	16	
5	17	18	19	20	

In [982]:
t.rename(1, 'name')
t

	'name'	'2'	'3'	'4'	
1	1	2	3	4	
2	5	6	7	8	
3	9	10	11	12	
4	13	14	15	16	
5	17	18	19	20	

In [983]:
t.rename('4', 'surname')
t

	'name'	'2'	'3'	'surname'	
1	1	2	3	4	
2	5	6	7	8	
3	9	10	11	12	
4	13	14	15	16	
5	17	18	19	20	

In [984]:
t.rename('name', 'name_v2')
t

	'name_v2'	'2'	'3'	'surname'	
1	1	2	3	4	
2	5	6	7	8	
3	9	10	11	12	
4	13	14	15	16	
5	17	18	19	20	

### Доступ к столбцам

In [985]:
t = Table([[ 1,  2,  3,  4],
           [ 5,  6,  7,  8],
           [ 9, 10, 11, 12],
           [13, 14, 15, 16],
           [17, 18, 19, 20]])
t.rename(1, 'name')
t.rename(4, 'surname')
t

	'name'	'2'	'3'	'surname'	
1	1	2	3	4	
2	5	6	7	8	
3	9	10	11	12	
4	13	14	15	16	
5	17	18	19	20	

#### Присвоение

In [986]:
t[1] = [0, 0, 0, 0, 0]
t

	'name'	'2'	'3'	'surname'	
1	0	2	3	4	
2	0	6	7	8	
3	0	10	11	12	
4	0	14	15	16	
5	0	18	19	20	

In [987]:
t['surname'] = [ 1 ] * len(t)
t

	'name'	'2'	'3'	'surname'	
1	0	2	3	1	
2	0	6	7	1	
3	0	10	11	1	
4	0	14	15	1	
5	0	18	19	1	

In [988]:
t[2] = list(range(16, 21))
t

	'name'	'2'	'3'	'surname'	
1	0	16	3	1	
2	0	17	7	1	
3	0	18	11	1	
4	0	19	15	1	
5	0	20	19	1	

#### Обращение

In [989]:
t['name']

	'name'	
1	0	
2	0	
3	0	
4	0	
5	0	

In [990]:
t[3]

	'3'	
1	3	
2	7	
3	11	
4	15	
5	19	

In [991]:
t[4]

	'surname'	
1	1	
2	1	
3	1	
4	1	
5	1	

### Функции `head` и `tail`

In [992]:
t = Table([[ 1,  2,  3,  4],
           [ 5,  6,  7,  8],
           [ 9, 10, 11, 12],
           [13, 14, 15, 16],
           [17, 18, 19, 20],
           [21, 22, 23, 24], 
           [25, 26, 27, 28]], names=['a', 'b', 'c', 'd'])
t

	'a'	'b'	'c'	'd'	
1	1	2	3	4	
2	5	6	7	8	
3	9	10	11	12	
4	13	14	15	16	
5	17	18	19	20	
6	21	22	23	24	
7	25	26	27	28	

In [993]:
t.head()

	'a'	'b'	'c'	'd'	
1	1	2	3	4	
2	5	6	7	8	
3	9	10	11	12	
4	13	14	15	16	
5	17	18	19	20	

In [994]:
t['a'].head(2)

	'a'	
1	1	
2	5	

In [995]:
t.tail()


	'a'	'b'	'c'	'd'	
3	9	10	11	12	
4	13	14	15	16	
5	17	18	19	20	
6	21	22	23	24	
7	25	26	27	28	

In [996]:
t[2].tail(3)

	'b'	
5	18	
6	22	
7	26	

### Методы `unique` и `count`

In [997]:
t = Table([[ i + j % 5 for i in range(4)] for j in range(10) ], names=['a', 'b', 'c', 'd'])
t

	'a'	'b'	'c'	'd'	
1	0	1	2	3	
2	1	2	3	4	
3	2	3	4	5	
4	3	4	5	6	
5	4	5	6	7	
6	0	1	2	3	
7	1	2	3	4	
8	2	3	4	5	
9	3	4	5	6	
10	4	5	6	7	

In [998]:
t.unique()

[0, 1, 2, 3, 4, 5, 6, 7]

In [999]:
t['a'].unique()

[0, 1, 2, 3, 4]

In [1000]:
t[4].unique()

[3, 4, 5, 6, 7]

In [1001]:
t.count()

	'count'	
a	10	
b	10	
c	10	
d	10	

In [1002]:
array = [ None ] * len(t)
array[5] = 10
array[2] = 100
t['b'] = array
t.count()

	'count'	
a	10	
b	2	
c	10	
d	10	

In [1003]:
t.count().unique()

[10, 2]

In [1004]:
t['b'].count()

	'count'	
b	2	

In [1005]:
list(t['b'].count())[0]

2